In [2]:
import pandas as pd
import googlemaps
from dotenv import dotenv_values

config = dotenv_values(".env")

gmaps_api_key = config["GOOGLE_MAPS_API_KEY"]
gmaps = googlemaps.Client(key=gmaps_api_key)

In [3]:
attractions = pd.read_csv('sample-data-Tourist_attractions.csv')
attractions.head()

,name,address,phone,email,lat,lng,url,country,state,city,star_count,rating_count,zip,primary_category_name,category_name
0,The Heart Of Europe - Dubai,The World Islands - Dubai - United Arab Emirates,+971 4 818 1481,sales@thoe.com,25.231606,55.161520,https://thoe.com/,United Arab Emirates,Dubai,Dubai,4.2,451,NaN,Tourist attractions,Tourist attractions
1,Al Jiyad Stables,R6FQ+H5R - Al Qudra Rd - Dubai - United Arab E...,+971 50 599 5866,info@aljiyadstables.com,24.823718,55.237696,http://www.aljiyadstables.com/,United Arab Emirates,Dubai,Dubai,4.6,490,NaN,Tourist attractions,Tourist attractions
2,Dubai Marina Cruise,Marina Plaza - offcie no 2192 - Dubai Marina -...,+971 56 324 1343,info@dubaidunebuggytours.com,25.075398,55.139825,http://dubaidunebuggytours.com/dhow-cruise-Mar...,United Arab Emirates,Dubai,Dubai,4.5,27,NaN,Tourist attractions,"Boat tour agencies,Cruise line companies,Touri..."
3,Dubai Butterfly Garden,"Al Barsha South 3, Dubailand Area, Beside Duba...",+971 4 422 8902,support@the-miracle-garden.com,25.061805,55.246561,http://www.dubaibutterflygarden.com/,United Arab Emirates,Dubai,Dubai,4.4,8812,NaN,Tourist attractions,"Tourist attractions,Gardens"
4,Sky Walker Yacht Tours,Dubai International Marine Club - Dubai - Unit...,+971 4 548 3302,charter@skywalker.ae,25.092188,55.142109,http://www.skywalker.ae/,United Arab Emirates,Dubai,Dubai,4.8,40,NaN,Tourist attractions,Tourist attractions


What does the google maps api return for a given location? 

In [4]:
attraction = attractions.iloc[0]
print(attraction['name'])
result = gmaps.find_place(input=attraction['name'], input_type='textquery', location_bias=f'point:{attraction["lat"]},{attraction["lng"]}')
result["candidates"]

The Heart Of Europe - Dubai


[{'place_id': 'ChIJl3hDbqFBXz4R-mt9u88gmIY'}]

The google maps api returns a list of places it thinks it could be. Since we provide a lat and lng it should probably return the exact location.

In [5]:
place_gmaps_id = result["candidates"][0]["place_id"]
#place_details = gmaps.place(place_id=place_gmaps_id, fields=['wheelchair_accessible_entrance'])
place_details = gmaps.place(place_id=place_gmaps_id)

place_details

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'The World Islands',
    'short_name': 'The World Islands',
    'types': ['sublocality_level_1', 'sublocality', 'political']},
   {'long_name': 'Dubai',
    'short_name': 'Dubai',
    'types': ['locality', 'political']},
   {'long_name': 'Dubai',
    'short_name': 'Dubai',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United Arab Emirates',
    'short_name': 'AE',
    'types': ['country', 'political']}],
  'adr_address': '<span class="street-address">The World Islands</span> - <span class="region">Dubai</span> - <span class="country-name">United Arab Emirates</span>',
  'business_status': 'OPERATIONAL',
  'current_opening_hours': {'open_now': True,
   'periods': [{'close': {'date': '2024-05-15',
      'day': 3,
      'time': '2359',
      'truncated': True},
     'open': {'date': '2024-05-09',
      'day': 4,
      'time': '0000',
      'truncated': True}}],
   'weekday_text': ['M

Some interesting things we can use to supplement our dataset:
- Opening hours
- Photos to show in the recommender
- We can update the reviews and ratings to be the most up to date
- We have the top 5 reviews whose text we might scrape to understand the place
- There's a types field with what type of attraction it is, that's slightly different from the category field we have
- There's a wheelchair_accessible field we can check for accessibility
- There's some additional atmospheric data such as serves brunch, etc which you can find out about here https://developers.google.com/maps/documentation/places/web-service/details

In [23]:
def get_place_details(place_id: str):
	place_details = gmaps.place(place_id = place_id, fields=['name', 'type', 'price_level', 'rating', 'user_ratings_total', 'wheelchair_accessible_entrance'])
	return place_details['result']
get_place_details(place_gmaps_id)

{'name': 'The Heart Of Europe - Dubai',
 'rating': 4.3,
 'types': ['tourist_attraction', 'point_of_interest', 'establishment'],
 'user_ratings_total': 579,
 'wheelchair_accessible_entrance': True}

In [ ]:
def get_place_id(name, lat, lng):
    result = gmaps.find_place(input=name, input_type='textquery', location_bias=f"point:{lat},{lng}")
    if result['status'] == 'OK' and len(result['candidates']) > 0:
        return result['candidates'][0]['place_id']
    return None

additional_details = []
for _, row in attractions.iterrows():
    place_id = get_place_id(row['name'], row['lat'], row['lng'])
    if place_id:
        details = get_place_details(place_id)
        additional_details.append(details)
    else:
        additional_details.append(None)

attractions['additional_details'] = additional_details